# HEADER

Import packages

In [1489]:
%%time

import pandas as pd
import numpy as np
import gapandas4 as gp

import os
import datetime

#from pandas import json_normalize
#import json

# Written functions
import dates_funcs

import importlib
importlib.reload(dates_funcs)

# to write to excel-file
import openpyxl


CPU times: user 1.05 ms, sys: 3.66 ms, total: 4.71 ms
Wall time: 3.11 ms


In [1490]:
%%time

os.getcwd()

CPU times: user 108 µs, sys: 127 µs, total: 235 µs
Wall time: 172 µs


'/Users/emil/miniforge3/envs/googleapi/Notebooks/Analytics'

In [1491]:
%%time

from IPython.display import display, HTML

# Function to fully display dataframe
def force_show_all(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
        display(HTML(df.to_html()))
# ... now when you're ready to fully display df:
#force_show_all(df)

CPU times: user 12 µs, sys: 31 µs, total: 43 µs
Wall time: 38.9 µs


Properties for connections

In [1492]:
%%time

property_id = "298727788"
credentials_json_path="/Users/emil/miniforge3/envs/googleapi/Notebooks/Analytics/client_secrets.json"

service_account = credentials_json_path

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 4.29 µs


Get metadata

In [1493]:
%%time

metadata = gp.get_metadata(service_account, property_id)
dfMetadata = pd.DataFrame(metadata)

CPU times: user 62.9 ms, sys: 17.8 ms, total: 80.7 ms
Wall time: 472 ms


In [1494]:
%%time

dfMetadata.head()

CPU times: user 63 µs, sys: 33 µs, total: 96 µs
Wall time: 94.9 µs


,Type,Data type,API Name,UI Name,Description,Custom definition
0,Dimension,STRING,achievementId,Achievement ID,The achievement ID in a game for an event. Pop...,False
1,Dimension,STRING,adFormat,Ad format,Describes the way ads looked and where they we...,False
2,Dimension,STRING,adSourceName,Ad source,The source network that served the ad. Typical...,False
3,Dimension,STRING,adUnitName,Ad unit,The name you chose to describe this Ad unit. A...,False
4,Dimension,STRING,appVersion,App version,The app's versionName (Android) or short bundl...,False


Find saved maxdate

In [1495]:
%%time

f = open("latestDate.txt")
maxSavedDate = f.read()
f.close()

print(type(maxSavedDate))

maxSavedDate = datetime.datetime.strptime(maxSavedDate,'%Y-%m-%d').date()

print(type(maxSavedDate))
print(maxSavedDate)

<class 'str'>
<class 'datetime.date'>
2023-01-14
CPU times: user 527 µs, sys: 1.04 ms, total: 1.57 ms
Wall time: 917 µs


Batch-queries

Dates to fetch data

In [1496]:
%%time

start_date = "2022-01-01"
end_date = "2022-01-31"

CPU times: user 2 µs, sys: 13 µs, total: 15 µs
Wall time: 16.2 µs


## Lists

In [1497]:
%%time

# Dimensions
dimensions = ["language","languageCode","browser","deviceCategory","mobileDeviceBranding",
              "mobileDeviceMarketingName","mobileDeviceModel","operatingSystemWithVersion",
              "platform","screenResolution","firstUserDefaultChannelGroup","firstUserSource",
              "sessionDefaultChannelGroup","sessionSource","eventName","pagePath","linkUrl",
              "landingPage","pageReferrer"]

# Does not work with cityId, only countryId
# does not work with dateHour only date.
dimensionsSpecial = ["userGender","brandingInterest"]

# Does not work with cityId, only countryId,
# does not work with dateHour only date:
# also cant request cityId and date in same request.
dimensionsSpecialSpecial = ["userAgeBracket"]

dateHourList = ["dateHour"]
dateList = ["date"]
cityIdList = ["cityId"]
countryIdList = ["countryId"]

# Metrics with empty first place and repeat last to count 1-18
metricList = ["","totalUsers","newUsers","activeUsers","userEngagementDuration","scrolledUsers",
              "averageSessionDuration","bounceRate","engagedSessions","engagementRate","sessions",
              "sessionsPerUser","eventCount","eventCountPerUser","eventsPerSession","screenPageViews",
              "screenPageViewsPerSession","screenPageViewsPerUser","totalUsers"]
    
loyaltyList = ["wauPerMau","dauPerMau","dauPerWau"]
    
activityList = ["active28DayUsers","active7DayUsers","activeUsers"]


CPU times: user 6 µs, sys: 24 µs, total: 30 µs
Wall time: 31.9 µs


## Requests

### dimensions

#### metricList

In [1498]:
%%time

# Empty list to fill with dataframes
dfs = [[],[],[],]

# Request for all dimensions in list
for x in range(len(dimensions)):
    for y, z in zip([1,7,13],[0,1,2]): 

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateHourList[0]),
                gp.Dimension(name=cityIdList[0]),
                gp.Dimension(name=dimensions[x])
            ],
            metrics=[
                gp.Metric(name=metricList[y]),
                gp.Metric(name=metricList[y+1]),
                gp.Metric(name=metricList[y+2]),
                gp.Metric(name=metricList[y+3]),
                gp.Metric(name=metricList[y+4]),
                gp.Metric(name=metricList[y+5]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs[z].append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 5.93 s, sys: 302 ms, total: 6.23 s
Wall time: 43.5 s


#### loyaltyList

In [1499]:
%%time

# Empty list to fill with dataframes
dfs_l = []

# Request for all dimensions in list
for x in range(len(dimensions)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=cityIdList[0]),
                gp.Dimension(name=dimensions[x])
            ],
            metrics=[
                gp.Metric(name=loyaltyList[0]),
                gp.Metric(name=loyaltyList[1]),
                gp.Metric(name=loyaltyList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_l.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 2.17 s, sys: 88.6 ms, total: 2.26 s
Wall time: 18.2 s


#### activityList

In [1500]:
%%time

# Empty list to fill with dataframes
dfs_a = []

# Request for all dimensions in list
for x in range(len(dimensions)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=cityIdList[0]),
                gp.Dimension(name=dimensions[x])
            ],
            metrics=[
                gp.Metric(name=activityList[0]),
                gp.Metric(name=activityList[1]),
                gp.Metric(name=activityList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_a.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 2.95 s, sys: 110 ms, total: 3.06 s
Wall time: 19.7 s


### dimensionsSpecial

In [1501]:
%%time

# Empty list to fill with dataframes
dfs_s = [[],[],[],]

# Request for all dimensions in list
for x in range(len(dimensionsSpecial)):
    for y, z in zip([1,7,13],[0,1,2]): 

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=countryIdList[0]),
                gp.Dimension(name=dimensionsSpecial[x])
            ],
            metrics=[
                gp.Metric(name=metricList[y]),
                gp.Metric(name=metricList[y+1]),
                gp.Metric(name=metricList[y+2]),
                gp.Metric(name=metricList[y+3]),
                gp.Metric(name=metricList[y+4]),
                gp.Metric(name=metricList[y+5]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_s[z].append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 391 ms, sys: 17.7 ms, total: 409 ms
Wall time: 3.56 s


#### loyaltyList

In [1502]:
%%time

# Empty list to fill with dataframes
dfs_s_l = []

# Request for all dimensions in list
for x in range(len(dimensionsSpecial)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=countryIdList[0]),
                gp.Dimension(name=dimensionsSpecial[x])
            ],
            metrics=[
                gp.Metric(name=loyaltyList[0]),
                gp.Metric(name=loyaltyList[1]),
                gp.Metric(name=loyaltyList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_s_l.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 127 ms, sys: 8.25 ms, total: 135 ms
Wall time: 1.29 s


#### activityList

In [1503]:
%%time

# Empty list to fill with dataframes
dfs_s_a = []

# Request for all dimensions in list
for x in range(len(dimensionsSpecial)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=countryIdList[0]),
                gp.Dimension(name=dimensionsSpecial[x])
            ],
            metrics=[
                gp.Metric(name=activityList[0]),
                gp.Metric(name=activityList[1]),
                gp.Metric(name=activityList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_s_a.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 162 ms, sys: 8.67 ms, total: 171 ms
Wall time: 1.05 s


### dimensionsSpecialSpecial date

#### metricList

In [1504]:
%%time

# Empty list to fill with dataframes
dfs_ssd = [[],[],[],]

# Request for all dimensions in list
for x in range(len(dimensionsSpecialSpecial)):
    for y, z in zip([1,7,13],[0,1,2]): 

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=dimensionsSpecialSpecial[x])
            ],
            metrics=[
                gp.Metric(name=metricList[y]),
                gp.Metric(name=metricList[y+1]),
                gp.Metric(name=metricList[y+2]),
                gp.Metric(name=metricList[y+3]),
                gp.Metric(name=metricList[y+4]),
                gp.Metric(name=metricList[y+5]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_ssd[z].append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 247 ms, sys: 11.8 ms, total: 259 ms
Wall time: 1.97 s


#### loyaltyList

In [1505]:
%%time

# Empty list to fill with dataframes
dfs_ssd_l = []

# Request for all dimensions in list
for x in range(len(dimensionsSpecialSpecial)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=dimensionsSpecialSpecial[x])
            ],
            metrics=[
                gp.Metric(name=loyaltyList[0]),
                gp.Metric(name=loyaltyList[1]),
                gp.Metric(name=loyaltyList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_ssd_l.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 76.3 ms, sys: 4.86 ms, total: 81.1 ms
Wall time: 884 ms


#### activityList

In [1506]:
%%time

# Empty list to fill with dataframes
dfs_ssd_a = []

# Request for all dimensions in list
for x in range(len(dimensionsSpecialSpecial)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=dateList[0]),
                gp.Dimension(name=dimensionsSpecialSpecial[x])
            ],
            metrics=[
                gp.Metric(name=activityList[0]),
                gp.Metric(name=activityList[1]),
                gp.Metric(name=activityList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_ssd_a.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 76.2 ms, sys: 5.28 ms, total: 81.5 ms
Wall time: 695 ms


### dimensionsSpecialSpecial countryId

#### metricList

In [1507]:
%%time

# Empty list to fill with dataframes
dfs_ssc = [[],[],[],]

# Request for all dimensions in list
for x in range(len(dimensionsSpecialSpecial)):
    for y, z in zip([1,7,13],[0,1,2]): 

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=countryIdList[0]),
                gp.Dimension(name=dimensionsSpecialSpecial[x])
            ],
            metrics=[
                gp.Metric(name=metricList[y]),
                gp.Metric(name=metricList[y+1]),
                gp.Metric(name=metricList[y+2]),
                gp.Metric(name=metricList[y+3]),
                gp.Metric(name=metricList[y+4]),
                gp.Metric(name=metricList[y+5]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_ssc[z].append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 251 ms, sys: 10.6 ms, total: 262 ms
Wall time: 2.24 s


#### loyaltyList

In [1508]:
%%time

# Empty list to fill with dataframes
dfs_ssc_l = []

# Request for all dimensions in list
for x in range(len(dimensionsSpecialSpecial)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=countryIdList[0]),
                gp.Dimension(name=dimensionsSpecialSpecial[x])
            ],
            metrics=[
                gp.Metric(name=loyaltyList[0]),
                gp.Metric(name=loyaltyList[1]),
                gp.Metric(name=loyaltyList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_ssc_l.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 79 ms, sys: 5.08 ms, total: 84.1 ms
Wall time: 488 ms


#### activityList

In [1509]:
%%time

# Empty list to fill with dataframes
dfs_ssc_a = []

# Request for all dimensions in list
for x in range(len(dimensionsSpecialSpecial)):

        report_request = gp.RunReportRequest(
            property=f"properties/{property_id}",
            dimensions=[
                gp.Dimension(name=countryIdList[0]),
                gp.Dimension(name=dimensionsSpecialSpecial[x])
            ],
            metrics=[
                gp.Metric(name=activityList[0]),
                gp.Metric(name=activityList[1]),
                gp.Metric(name=activityList[2]),
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
        # Perform query and append to list
        dfs_ssc_a.append(gp.query(service_account, report_request, report_type="report"))

CPU times: user 78.1 ms, sys: 4.8 ms, total: 82.9 ms
Wall time: 495 ms


## Formatting

#### metricList

In [1510]:
%%time

# Drop the extra Totalusers column, since it exists twice in the tables to merge
# since metricList contained it twice.

for x in range(len(dimensions)):
    for z in [2]:
        dfs[z][x].drop(columns=['totalUsers'],inplace=True)

for x in range(len(dimensionsSpecial)):
    for z in [2]:
        dfs_s[z][x].drop(columns=['totalUsers'],inplace=True)

for x in range(len(dimensionsSpecialSpecial)):
    for z in [2]:
        dfs_ssd[z][x].drop(columns=['totalUsers'],inplace=True)

for x in range(len(dimensionsSpecialSpecial)):
    for z in [2]:
        dfs_ssc[z][x].drop(columns=['totalUsers'],inplace=True)

CPU times: user 11.2 ms, sys: 1.82 ms, total: 13 ms
Wall time: 11.9 ms


#### metricList

In [1511]:
%%time

for x in range(len(dimensions)):
    for z in [0,1,2]: 
        dfs[z][x] = dates_funcs.sortOut_dateHour_short(dfs[z][x])

for x in range(len(dimensionsSpecial)):
    for z in [0,1,2]: 
        dfs_s[z][x] = dates_funcs.sortOut_date_short(dfs_s[z][x])
        
for x in range(len(dimensionsSpecialSpecial)):
    for z in [0,1,2]: 
        dfs_ssd[z][x] = dates_funcs.sortOut_date_short(dfs_ssd[z][x])

CPU times: user 220 ms, sys: 5.09 ms, total: 225 ms
Wall time: 224 ms


#### loyaltyList

In [1512]:
%%time

for x in range(len(dimensions)):
        dfs_l[x] = dates_funcs.sortOut_date_short(dfs_l[x])

for x in range(len(dimensionsSpecial)):
        dfs_s_l[x] = dates_funcs.sortOut_date_short(dfs_s_l[x])
        
for x in range(len(dimensionsSpecialSpecial)):
        dfs_ssd_l[x] = dates_funcs.sortOut_date_short(dfs_ssd_l[x])

CPU times: user 58.5 ms, sys: 2.66 ms, total: 61.2 ms
Wall time: 61.2 ms


#### activityList

In [1513]:
%%time

for x in range(len(dimensions)):
        dfs_a[x] = dates_funcs.sortOut_date_short(dfs_a[x])

for x in range(len(dimensionsSpecial)):
        dfs_s_a[x] = dates_funcs.sortOut_date_short(dfs_s_a[x])
        
for x in range(len(dimensionsSpecialSpecial)):
        dfs_ssd_a[x] = dates_funcs.sortOut_date_short(dfs_ssd_a[x])

CPU times: user 79.5 ms, sys: 2.77 ms, total: 82.3 ms
Wall time: 82.1 ms


## Merge

In [1514]:
%%time

dfs_merged = list(range(len(dimensions)))
for x in range(len(dimensions)):
    mergeOn = ['dateTime','dateFull','time','cityId'] 
    mergeDim = [dimensions[x]]
    mergeOn =  mergeOn + mergeDim
    dfs_merged[x] = dfs[0][x].merge(dfs[1][x], how='outer', on=mergeOn)

CPU times: user 26.8 ms, sys: 2.41 ms, total: 29.2 ms
Wall time: 28.2 ms


In [1515]:
%%time

for n in range(len(dimensions)):
    mergeOn = ['dateTime','dateFull','time','cityId'] 
    mergeDim = [dimensions[n]]
    mergeOn =  mergeOn + mergeDim
    dfs_merged[n] = dfs_merged[n].merge(dfs[2][n], how='outer', on=mergeOn)

CPU times: user 26.2 ms, sys: 2.25 ms, total: 28.5 ms
Wall time: 27.3 ms


In [1516]:
dfs_s_merged = list(range(len(dimensionsSpecial)))
for x in range(len(dimensionsSpecial)):
    mergeOn = ['dateFull','countryId'] 
    mergeDim = [dimensionsSpecial[x]]
    mergeOn =  mergeOn + mergeDim
    dfs_s_merged[x] = dfs_s[0][x].merge(dfs_s[1][x], how='outer', on=mergeOn)

In [1517]:
for n in range(len(dimensionsSpecial)):
    mergeOn = ['dateFull','countryId'] 
    mergeDim = [dimensionsSpecial[n]]
    mergeOn =  mergeOn + mergeDim
    dfs_s_merged[n] = dfs_s_merged[n].merge(dfs_s[2][n], how='outer', on=mergeOn)

In [1518]:
dfs_ssd_merged = list(range(len(dimensionsSpecialSpecial)))
for x in range(len(dimensionsSpecialSpecial)):
    mergeOn = ['dateFull'] 
    mergeDim = [dimensionsSpecialSpecial[x]]
    mergeOn =  mergeOn + mergeDim
    dfs_ssd_merged[x] = dfs_ssd[0][x].merge(dfs_ssd[1][x], how='outer', on=mergeOn)

In [1519]:
for n in range(len(dimensionsSpecialSpecial)):
    mergeOn = ['dateFull'] 
    mergeDim = [dimensionsSpecialSpecial[n]]
    mergeOn =  mergeOn + mergeDim
    dfs_ssd_merged[n] = dfs_ssd_merged[n].merge(dfs_ssd[2][n], how='outer', on=mergeOn)

In [1520]:
dfs_ssc_merged = list(range(len(dimensionsSpecialSpecial)))
for x in range(len(dimensionsSpecialSpecial)):
    mergeOn = ['countryId'] 
    mergeDim = [dimensionsSpecialSpecial[x]]
    mergeOn =  mergeOn + mergeDim
    dfs_ssc_merged[x] = dfs_ssc[0][x].merge(dfs_ssc[1][x], how='outer', on=mergeOn)

In [1521]:
for n in range(len(dimensionsSpecialSpecial)):
    mergeOn = ['countryId'] 
    mergeDim = [dimensionsSpecialSpecial[n]]
    mergeOn =  mergeOn + mergeDim
    dfs_ssc_merged[n] = dfs_ssc_merged[n].merge(dfs_ssc[2][n], how='outer', on=mergeOn)

## Sort out NA

#### metricList

In [1522]:
dfs_merged_fillNA = list(range(len(dimensions)))

for n in range(len(dimensions)):
    dfs_merged_fillNA[n] = dfs_merged[n].replace('', np.nan)
    dfs_merged_fillNA[n] = dfs_merged_fillNA[n].replace('(not set)', np.nan)
    print(dfs_merged_fillNA[n].shape)

(516, 22)
(540, 22)
(553, 22)
(537, 22)
(558, 22)
(533, 22)
(576, 22)
(579, 22)
(497, 22)
(594, 22)
(528, 22)
(545, 22)
(525, 22)
(543, 22)
(1786, 22)
(796, 22)
(536, 22)
(591, 22)
(744, 22)


In [1523]:
dfs_s_merged_fillNA = list(range(len(dimensionsSpecial)))

for n in range(len(dimensionsSpecial)):
    dfs_s_merged_fillNA[n] = dfs_s_merged[n].replace('', np.nan)
    dfs_s_merged_fillNA[n] = dfs_s_merged_fillNA[n].replace('(not set)', np.nan)
    print(dfs_s_merged_fillNA[n].shape)

(11, 20)
(42, 20)


In [1524]:
dfs_ssd_merged_fillNA = list(range(len(dimensionsSpecialSpecial)))

for n in range(len(dimensionsSpecialSpecial)):
    dfs_ssd_merged_fillNA[n] = dfs_ssd_merged[n].replace('', np.nan)
    dfs_ssd_merged_fillNA[n] = dfs_ssd_merged_fillNA[n].replace('(not set)', np.nan)
    print(dfs_ssd_merged_fillNA[n].shape)

(3, 19)


In [1525]:
dfs_ssc_merged_fillNA = list(range(len(dimensionsSpecialSpecial)))

for n in range(len(dimensionsSpecialSpecial)):
    dfs_ssc_merged_fillNA[n] = dfs_ssc_merged[n].replace('', np.nan)
    dfs_ssc_merged_fillNA[n] = dfs_ssc_merged_fillNA[n].replace('(not set)', np.nan)
    print(dfs_ssc_merged_fillNA[n].shape)

(7, 19)


#### loyaltyList

In [1526]:
dfs_l_fillNA = list(range(len(dimensions)))

for n in range(len(dimensions)):
    dfs_l_fillNA[n] = dfs_l[n].replace('', np.nan)
    dfs_l_fillNA[n] = dfs_l_fillNA[n].replace('(not set)', np.nan)
    print(dfs_l_fillNA[n].shape)

(954, 6)
(1119, 6)
(1192, 6)
(1047, 6)
(1281, 6)
(1303, 6)
(1674, 6)
(1863, 6)
(809, 6)
(2054, 6)
(1013, 6)
(1141, 6)
(1019, 6)
(1154, 6)
(3478, 6)
(2748, 6)
(994, 6)
(2122, 6)
(2100, 6)


In [1527]:
dfs_s_l_fillNA = list(range(len(dimensionsSpecial)))

for n in range(len(dimensionsSpecial)):
    dfs_s_l_fillNA[n] = dfs_s_l[n].replace('', np.nan)
    dfs_s_l_fillNA[n] = dfs_s_l_fillNA[n].replace('(not set)', np.nan)
    print(dfs_s_l_fillNA[n].shape)

(11, 6)
(42, 6)


In [1528]:
dfs_ssd_l_fillNA = list(range(len(dimensionsSpecialSpecial)))

for n in range(len(dimensionsSpecialSpecial)):
    dfs_ssd_l_fillNA[n] = dfs_ssd_l[n].replace('', np.nan)
    dfs_ssd_l_fillNA[n] = dfs_ssd_l_fillNA[n].replace('(not set)', np.nan)
    print(dfs_ssd_l_fillNA[n].shape)

(3, 5)


In [1529]:
dfs_ssc_l_fillNA = list(range(len(dimensionsSpecialSpecial)))

for n in range(len(dimensionsSpecialSpecial)):
    dfs_ssc_l_fillNA[n] = dfs_ssc_l[n].replace('', np.nan)
    dfs_ssc_l_fillNA[n] = dfs_ssc_l_fillNA[n].replace('(not set)', np.nan)
    print(dfs_ssc_l_fillNA[n].shape)

(7, 5)


#### activityList

In [1530]:
dfs_a_fillNA = list(range(len(dimensions)))

for n in range(len(dimensions)):
    dfs_a_fillNA[n] = dfs_a[n].replace('', np.nan)
    dfs_a_fillNA[n] = dfs_a_fillNA[n].replace('(not set)', np.nan)
    print(dfs_a_fillNA[n].shape)

(2011, 6)
(2501, 6)
(2608, 6)
(2207, 6)
(2922, 6)
(3076, 6)
(4234, 6)
(4525, 6)
(1565, 6)
(5295, 6)
(2006, 6)
(2439, 6)
(2022, 6)
(2460, 6)
(6841, 6)
(6653, 6)
(2063, 6)
(5306, 6)
(4952, 6)


In [1531]:
dfs_s_a_fillNA = list(range(len(dimensionsSpecial)))

for n in range(len(dimensionsSpecial)):
    dfs_s_a_fillNA[n] = dfs_s_a[n].replace('', np.nan)
    dfs_s_a_fillNA[n] = dfs_s_a_fillNA[n].replace('(not set)', np.nan)
    print(dfs_s_a_fillNA[n].shape)

(11, 6)
(42, 6)


In [1532]:
dfs_ssd_a_fillNA = list(range(len(dimensionsSpecialSpecial)))

for n in range(len(dimensionsSpecialSpecial)):
    dfs_ssd_a_fillNA[n] = dfs_ssd_a[n].replace('', np.nan)
    dfs_ssd_a_fillNA[n] = dfs_ssd_a_fillNA[n].replace('(not set)', np.nan)
    print(dfs_ssd_a_fillNA[n].shape)

(3, 5)


In [1533]:
dfs_ssc_a_fillNA = list(range(len(dimensionsSpecialSpecial)))

for n in range(len(dimensionsSpecialSpecial)):
    dfs_ssc_a_fillNA[n] = dfs_ssc_a[n].replace('', np.nan)
    dfs_ssc_a_fillNA[n] = dfs_ssc_a_fillNA[n].replace('(not set)', np.nan)
    print(dfs_ssc_a_fillNA[n].shape)

(7, 5)


## Max time to latestDatefile

In [1534]:
%%time

# Find largest date in dataframes, looking at df eventName, which should have all dates. (?)
maxTimestamp = dfs_merged_fillNA[14]["dateFull"].max()
maxTimestampString = datetime.datetime.strftime(maxTimestamp,'%Y-%m-%d')
maxDate = datetime.datetime.strptime(maxTimestampString,'%Y-%m-%d').date()

print(maxDate)

2023-01-14
CPU times: user 231 µs, sys: 21 µs, total: 252 µs
Wall time: 250 µs


In [1535]:
%%time 

# If date is larger than priviously max date.
if maxDate > maxSavedDate:
# Write largest date to file
    f = open("latestDate.txt", 'w')
    f.write(maxTimestampString)
    f.close()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.29 µs


## Dataframes to .csv

In [1536]:
csvFilePath = "output/csv/"
fileName = dimensions[n]

sep = ","

def appendDFToCSV(df, csvFilePath, fileName, sep):

        fileNameFull = csvFilePath + fileName
        print(fileNameFull)
        if not os.path.isfile(fileNameFull):
            df.to_csv(fileNameFull, mode='a', index=0, sep=sep)
        elif len(df.columns) != len(pd.read_csv(fileNameFull, nrows=1, sep=sep).columns):
            raise Exception("Columns do not match!! Dataframe has " + str(len(df.columns)) + " columns. CSV file has " + str(len(pd.read_csv(fileNameFull, nrows=1, sep=sep).columns)) + " columns.")
        #elif not (df.columns == pd.read_csv(fileNameFull, nrows=1, sep=sep).columns).all():
        #    raise Exception("Columns and column order of dataframe and csv file do not match!!")
        else:
            df.to_csv(fileNameFull, mode='a', index=0, sep=sep, header=False)


In [1537]:
for n in range(len(dfs_merged_fillNA)):
    csvFilePath = "output/csv/"
    fileName = dimensions[n]+"_dateHour"+"_cityId"+".csv" 
    appendDFToCSV(dfs_merged_fillNA[n], csvFilePath, dimensions[n]+"_dateHour"+"_cityId"+".csv", sep)

output/csv/language_dateHour_cityId.csv
output/csv/languageCode_dateHour_cityId.csv
output/csv/browser_dateHour_cityId.csv
output/csv/deviceCategory_dateHour_cityId.csv
output/csv/mobileDeviceBranding_dateHour_cityId.csv
output/csv/mobileDeviceMarketingName_dateHour_cityId.csv
output/csv/mobileDeviceModel_dateHour_cityId.csv
output/csv/operatingSystemWithVersion_dateHour_cityId.csv
output/csv/platform_dateHour_cityId.csv
output/csv/screenResolution_dateHour_cityId.csv
output/csv/firstUserDefaultChannelGroup_dateHour_cityId.csv
output/csv/firstUserSource_dateHour_cityId.csv
output/csv/sessionDefaultChannelGroup_dateHour_cityId.csv
output/csv/sessionSource_dateHour_cityId.csv
output/csv/eventName_dateHour_cityId.csv
output/csv/pagePath_dateHour_cityId.csv
output/csv/linkUrl_dateHour_cityId.csv
output/csv/landingPage_dateHour_cityId.csv
output/csv/pageReferrer_dateHour_cityId.csv


In [1538]:
for n in range(len(dfs_s_merged_fillNA)):
    csvFilePath = "output/csv/"
    fileName = dimensionsSpecial[n]+"_date"+"_countryId"+".csv"
    appendDFToCSV(dfs_s_merged_fillNA[n], csvFilePath, dimensionsSpecial[n]+"_date"+"_countryId"+".csv", sep)

output/csv/userGender_date_countryId.csv
output/csv/brandingInterest_date_countryId.csv


In [1539]:
for n in range(len(dfs_ssd_merged_fillNA)):
    csvFilePath = "output/csv/"
    fileName = dimensionsSpecialSpecial[n]+"_date"".csv"
    appendDFToCSV(dfs_ssd_merged_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_date"+".csv", sep)

output/csv/userAgeBracket_date.csv


In [1540]:
for n in range(len(dfs_ssc_merged_fillNA)):
    csvFilePath = "output/csv/"
    fileName = dimensionsSpecialSpecial[n]+"_countryId"+".csv"
    appendDFToCSV(dfs_ssc_merged_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_countryId"+".csv", sep)

output/csv/userAgeBracket_countryId.csv


In [1541]:
dfs_ssc_merged_fillNA[0]

,countryId,userAgeBracket,totalUsers,newUsers,activeUsers,userEngagementDuration,scrolledUsers,averageSessionDuration,bounceRate,engagedSessions,engagementRate,sessions,sessionsPerUser,eventCount,eventCountPerUser,eventsPerSession,screenPageViews,screenPageViewsPerSession,screenPageViewsPerUser
0,SE,unknown,315,309,311,25065,88,145.65285277207977,0.3504273504273504,228,0.6495726495726496,351,1.1286173633440515,1707,5.4887459807073951,4.8632478632478628,521,1.4843304843304843,1.67524115755627
1,SE,25-34,30,26,29,2684,9,173.2595374,0.55,18,0.45,40,1.3793103448275863,217,7.4827586206896548,5.425,73,1.825,2.5172413793103448
2,SE,45-54,26,23,26,1421,9,133.295692,0.29032258064516131,22,0.70967741935483875,31,1.1923076923076923,153,5.884615384615385,4.935483870967742,46,1.4838709677419355,1.7692307692307692
3,SE,65+,23,18,22,1593,12,126.62616813793103,0.34482758620689657,19,0.65517241379310343,29,1.3181818181818181,122,5.5454545454545459,4.2068965517241379,35,1.2068965517241379,1.5909090909090908
4,SE,35-44,19,17,19,1203,7,267.81998942307695,0.73076923076923073,7,0.26923076923076922,26,1.368421052631579,100,5.2631578947368425,3.8461538461538463,28,1.0769230769230769,1.4736842105263157
5,SE,55-64,18,17,17,645,6,43.394766040000007,0.6,10,0.4,25,1.4705882352941178,111,6.5294117647058822,4.44,33,1.32,1.9411764705882353
6,SE,18-24,10,7,9,372,4,47.170161833333331,0.41666666666666669,7,0.58333333333333337,12,1.3333333333333333,57,6.333333333333333,4.75,18,1.5,2


In [1542]:
for n in range(len(dfs_l_fillNA)):
    csvFilePath = "output/csv/loyalty/"
    fileName = dimensions[n]+"_dateHour"+"_cityId"+"_loyalty"+".csv"
    appendDFToCSV(dfs_l_fillNA[n], csvFilePath, dimensions[n]+"_dateHour"+"_cityId"+"_loyalty"+".csv", sep)

output/csv/loyalty/language_dateHour_cityId_loyalty.csv
output/csv/loyalty/languageCode_dateHour_cityId_loyalty.csv
output/csv/loyalty/browser_dateHour_cityId_loyalty.csv
output/csv/loyalty/deviceCategory_dateHour_cityId_loyalty.csv
output/csv/loyalty/mobileDeviceBranding_dateHour_cityId_loyalty.csv
output/csv/loyalty/mobileDeviceMarketingName_dateHour_cityId_loyalty.csv
output/csv/loyalty/mobileDeviceModel_dateHour_cityId_loyalty.csv
output/csv/loyalty/operatingSystemWithVersion_dateHour_cityId_loyalty.csv
output/csv/loyalty/platform_dateHour_cityId_loyalty.csv
output/csv/loyalty/screenResolution_dateHour_cityId_loyalty.csv
output/csv/loyalty/firstUserDefaultChannelGroup_dateHour_cityId_loyalty.csv
output/csv/loyalty/firstUserSource_dateHour_cityId_loyalty.csv
output/csv/loyalty/sessionDefaultChannelGroup_dateHour_cityId_loyalty.csv
output/csv/loyalty/sessionSource_dateHour_cityId_loyalty.csv
output/csv/loyalty/eventName_dateHour_cityId_loyalty.csv
output/csv/loyalty/pagePath_dateHour

In [1543]:
for n in range(len(dfs_s_l_fillNA)):
    csvFilePath = "output/csv/loyalty/"
    fileName = dimensionsSpecial[n]+"_date"+"_countryId"+"_loyalty"+".csv"
    appendDFToCSV(dfs_s_l_fillNA[n], csvFilePath, dimensionsSpecial[n]+"_date"+"_countryId"+"_loyalty"+".csv", sep)

output/csv/loyalty/userGender_date_countryId_loyalty.csv
output/csv/loyalty/brandingInterest_date_countryId_loyalty.csv


In [1544]:
for n in range(len(dfs_ssd_l_fillNA)):
    csvFilePath = "output/csv/loyalty/"
    fileName = dimensionsSpecialSpecial[n]+"_date"+"_loyalty"+".csv"
    appendDFToCSV(dfs_ssd_l_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_date"+"_loyalty"+".csv", sep)

output/csv/loyalty/userAgeBracket_date_loyalty.csv


In [1545]:
for n in range(len(dfs_ssc_l_fillNA)):
    csvFilePath = "output/csv/loyalty/"
    fileName = dimensionsSpecialSpecial[n]+"_countryId"+"_loyalty"+".csv"
    appendDFToCSV(dfs_ssc_l_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_countryId"+"_loyalty"+".csv", sep)

output/csv/loyalty/userAgeBracket_countryId_loyalty.csv


In [1546]:
for n in range(len(dfs_a_fillNA)):
    csvFilePath = "output/csv/activity/"
    fileName = dimensions[n]+"_dateHour"+"_cityId"+"_activity"+".csv"
    appendDFToCSV(dfs_a_fillNA[n], csvFilePath, dimensions[n]+"_dateHour"+"_cityId"+"_activity"+".csv", sep)

output/csv/activity/language_dateHour_cityId_activity.csv
output/csv/activity/languageCode_dateHour_cityId_activity.csv
output/csv/activity/browser_dateHour_cityId_activity.csv
output/csv/activity/deviceCategory_dateHour_cityId_activity.csv
output/csv/activity/mobileDeviceBranding_dateHour_cityId_activity.csv
output/csv/activity/mobileDeviceMarketingName_dateHour_cityId_activity.csv
output/csv/activity/mobileDeviceModel_dateHour_cityId_activity.csv
output/csv/activity/operatingSystemWithVersion_dateHour_cityId_activity.csv
output/csv/activity/platform_dateHour_cityId_activity.csv
output/csv/activity/screenResolution_dateHour_cityId_activity.csv
output/csv/activity/firstUserDefaultChannelGroup_dateHour_cityId_activity.csv
output/csv/activity/firstUserSource_dateHour_cityId_activity.csv
output/csv/activity/sessionDefaultChannelGroup_dateHour_cityId_activity.csv
output/csv/activity/sessionSource_dateHour_cityId_activity.csv
output/csv/activity/eventName_dateHour_cityId_activity.csv
output

In [1547]:
for n in range(len(dfs_s_a_fillNA)):
    csvFilePath = "output/csv/activity/"
    fileName = dimensionsSpecial[n]+"_date"+"_countryId"+"_activity"+".csv"
    appendDFToCSV(dfs_s_a_fillNA[n], csvFilePath, dimensionsSpecial[n]+"_date"+"_countryId"+"_activity"+".csv", sep)

output/csv/activity/userGender_date_countryId_activity.csv
output/csv/activity/brandingInterest_date_countryId_activity.csv


In [1548]:
for n in range(len(dfs_ssd_a_fillNA)):
    csvFilePath = "output/csv/activity/"
    fileName = dimensionsSpecialSpecial[n]+"_date"+"_activity"+".csv"
    appendDFToCSV(dfs_ssd_a_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_date"+"_activity"+".csv", sep)

output/csv/activity/userAgeBracket_date_activity.csv


In [1549]:
for n in range(len(dfs_ssc_a_fillNA)):
    csvFilePath = "output/csv/activity/"
    fileName = dimensionsSpecialSpecial[n]+"_countryId"+"_activity"+".csv"
    appendDFToCSV(dfs_ssc_a_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_countryId"+"_activity"+".csv", sep)

output/csv/activity/userAgeBracket_countryId_activity.csv


## Date and Geography tables

In [1550]:
%%time

#start_date = "2022-01-01"
#end_date = "2022-01-31"

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs


In [1551]:
%%time

# Geography List
geoList = ['continentID','continent','countryId','country','region','cityId','city']

# Empty list to fill with dataframes

report_request = gp.RunReportRequest(
    property=f"properties/{property_id}",
     dimensions=[
         gp.Dimension(name=geoList[0]),
         gp.Dimension(name=geoList[1]),
         gp.Dimension(name=geoList[2]),
         gp.Dimension(name=geoList[3]),
         gp.Dimension(name=geoList[4]),
         gp.Dimension(name=geoList[5]),
         gp.Dimension(name=geoList[6]),
       ],
       metrics=[
        ],
        date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
    )
    
# Perform query and append to list
df_geo = gp.query(service_account, report_request, report_type="report")

CPU times: user 55 ms, sys: 3.69 ms, total: 58.7 ms
Wall time: 386 ms


In [1552]:
%%time

# dateList is all dimensions
dateList = ["dateHour"]

# Empty list to fill with dataframes

report_request = gp.RunReportRequest(
    property=f"properties/{property_id}",
     dimensions=[
         gp.Dimension(name=dateList[0]),
       ],
       metrics=[
        ],
        date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
    )

# Perform query and append to list
df_dateHour = gp.query(service_account, report_request, report_type="report")

CPU times: user 75.6 ms, sys: 4.22 ms, total: 79.8 ms
Wall time: 688 ms


In [1553]:
df_dateHour = dates_funcs.sortOut_dateHour(df_dateHour)

In [1554]:
df_geo

,continentID,continent,countryId,country,region,cityId,city
0,002,Africa,KE,Kenya,Nairobi County,1009824,Nairobi
1,002,Africa,TZ,Tanzania,Unguja North Region,(not set),(not set)
2,009,Oceania,AU,Australia,New South Wales,1000594,Central Coast
3,019,Americas,US,United States,(not set),(not set),(not set)
4,019,Americas,US,United States,Hawaii,1015618,Wailuku
...,...,...,...,...,...,...,...
132,150,Europe,SE,Sweden,Vastra Gotaland County,1012541,Lysekil
133,150,Europe,SE,Sweden,Vastra Gotaland County,1012566,Stenungsund
134,150,Europe,SE,Sweden,Vastra Gotaland County,1012568,Svenljunga
135,150,Europe,SE,Sweden,Vastra Gotaland County,1012581,Vanersborg


In [1555]:
df_dateHour

,dateTime,dateFull,year,quarter,month,monthName,week,weekday,weekdayName,time,hour
0,2023-01-08 02:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,02:00:00,2
1,2023-01-08 08:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,08:00:00,8
2,2023-01-08 09:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,09:00:00,9
3,2023-01-08 10:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,10:00:00,10
4,2023-01-08 11:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,11:00:00,11
...,...,...,...,...,...,...,...,...,...,...,...
141,2023-01-14 19:00:00,2023-01-14,2023,1,1,January,2,6,Saturday,19:00:00,19
142,2023-01-14 20:00:00,2023-01-14,2023,1,1,January,2,6,Saturday,20:00:00,20
143,2023-01-14 21:00:00,2023-01-14,2023,1,1,January,2,6,Saturday,21:00:00,21
144,2023-01-14 22:00:00,2023-01-14,2023,1,1,January,2,6,Saturday,22:00:00,22


In [1556]:
df_geoTest = df_geo.replace('', np.nan)
df_geoTest = df_geoTest.replace('(not set)', np.nan)
print(df_geoTest.shape)

(137, 7)


In [1557]:
df_dateHourTest = df_dateHour.replace('', np.nan)
df_dateHourTest = df_dateHourTest.replace('(not set)', np.nan)
print(df_dateHourTest.shape)

(146, 11)


In [1558]:
df_geoTest.head(30)

,continentID,continent,countryId,country,region,cityId,city
0,002,Africa,KE,Kenya,Nairobi County,1009824,Nairobi
1,002,Africa,TZ,Tanzania,Unguja North Region,NaN,NaN
2,009,Oceania,AU,Australia,New South Wales,1000594,Central Coast
3,019,Americas,US,United States,NaN,NaN,NaN
4,019,Americas,US,United States,Hawaii,1015618,Wailuku
5,019,Americas,US,United States,New Jersey,1022418,South Plainfield
6,019,Americas,US,United States,New York,1023191,New York
7,019,Americas,US,United States,North Carolina,1021111,Forest City
8,019,Americas,US,United States,Ohio,1023616,Celina
9,019,Americas,US,United States,Ohio,1023640,Columbus


In [1559]:
df_dateHourTest.head(30)

,dateTime,dateFull,year,quarter,month,monthName,week,weekday,weekdayName,time,hour
0,2023-01-08 02:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,02:00:00,2
1,2023-01-08 08:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,08:00:00,8
2,2023-01-08 09:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,09:00:00,9
3,2023-01-08 10:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,10:00:00,10
4,2023-01-08 11:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,11:00:00,11
5,2023-01-08 12:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,12:00:00,12
6,2023-01-08 13:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,13:00:00,13
7,2023-01-08 14:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,14:00:00,14
8,2023-01-08 15:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,15:00:00,15
9,2023-01-08 16:00:00,2023-01-08,2023,1,1,January,1,7,Sunday,16:00:00,16


In [1560]:
csvFilePath = "output/csv/reference/"
fileName = "geographyInfo.csv"
appendDFToCSV(df_geoTest, csvFilePath, fileName, sep)

output/csv/reference/geographyInfo.csv


In [1561]:
csvFilePath = "output/csv/reference/"
fileName = "dateInfo.csv"
appendDFToCSV(df_dateHourTest, csvFilePath, fileName, sep)

output/csv/reference/dateInfo.csv


## Table with all dates, not just those with fetched data, as reference

In [1562]:
# Date range
dates = pd.date_range('2022-01-01', '2030-01-01', freq="H",inclusive='left')
print(dates)

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 01:00:00',
               '2022-01-01 02:00:00', '2022-01-01 03:00:00',
               '2022-01-01 04:00:00', '2022-01-01 05:00:00',
               '2022-01-01 06:00:00', '2022-01-01 07:00:00',
               '2022-01-01 08:00:00', '2022-01-01 09:00:00',
               ...
               '2029-12-31 14:00:00', '2029-12-31 15:00:00',
               '2029-12-31 16:00:00', '2029-12-31 17:00:00',
               '2029-12-31 18:00:00', '2029-12-31 19:00:00',
               '2029-12-31 20:00:00', '2029-12-31 21:00:00',
               '2029-12-31 22:00:00', '2029-12-31 23:00:00'],
              dtype='datetime64[ns]', length=70128, freq='H')


In [1563]:
# List to Dataframe
dateReference = pd.DataFrame(dates, columns=['dateHour'])

In [1564]:
# Running function sortOut_dateHour
dateReference = dates_funcs.sortOut_dateHour(dateReference)

In [1565]:
csvFilePath = "output/csv/reference/"
fileName = "dateReference.csv"
fileNameFull = csvFilePath + fileName

# If file does not alreay exist, write to file
if not os.path.isfile(fileNameFull):
    dateReference.to_csv(fileNameFull, mode='w', index=0, sep=sep)

In [1566]:
print("FINISHED")

FINISHED


In [1116]:
fileNameRead = "geographyMaster.csv"
fileNameWrite = "geographyMaster.xlsx"
fullFilenameRead = csvFilePath + fileNameRead
fullFilenameWrite = csvFilePath + fileNameWrite

read_file = pd.read_csv(fullFilenameRead)
read_file.to_excel (fullFilenameWrite, index = None, header=True)

TypeError: NDFrame.to_excel() got an unexpected keyword argument 'mode'

In [1113]:
fileNameRead = "dateInfoMaster.csv"
fileNameWrite = "dateInfoMaster.xlsx"
fullFilenameRead = csvFilePath + fileNameRead
fullFilenameWrite = csvFilePath + fileNameWrite

read_file = pd.read_csv(fullFilenameRead)
read_file.to_excel (fullFilenameWrite, index = None, header=True)

In [1114]:
fileNameRead = "browser_dateHour_cityId.csv"
fileNameWrite = "browser_dateHour_cityId.xlsx"
fullFilenameRead = csvFilePath + fileNameRead
fullFilenameWrite = csvFilePath + fileNameWrite

read_file = pd.read_csv(fullFilenameRead)
read_file.to_excel (fullFilenameWrite, index = None, header=True)

In [1115]:
fileNameRead = "mobileDeviceMarketingName_dateHour_cityId.csv"
fileNameWrite = "mobileDeviceMarketingName_dateHour_cityId.xlsx"
fullFilenameRead = csvFilePath + fileNameRead
fullFilenameWrite = csvFilePath + fileNameWrite

read_file = pd.read_csv(fullFilenameRead)
read_file.to_excel (fullFilenameWrite, index = None, header=True)

In [1137]:
dfs_merged_fillNA[5]

,dateFull,time,cityId,mobileDeviceMarketingName,totalUsers,newUsers,activeUsers,userEngagementDuration,scrolledUsers,averageSessionDuration,...,engagedSessions,engagementRate,sessions,sessionsPerUser,eventCount,eventCountPerUser,eventsPerSession,screenPageViews,screenPageViewsPerSession,screenPageViewsPerUser
0,2023-01-08,02:00:00,1012296,NaN,1,0,0,0,0,0,...,0,0,1,0,2,0,2,1,1,0
1,2023-01-08,08:00:00,1012475,NaN,1,1,1,97,0,101.177543,...,1,1,1,1,6,6,6,2,2,2
2,2023-01-08,09:00:00,1012228,NaN,1,1,1,55,0,62.339947,...,1,1,1,1,8,8,8,3,3,3
3,2023-01-08,09:00:00,9067819,Galaxy S20+ 5G,1,1,1,110,0,110.709899,...,1,1,1,1,4,4,4,1,1,1
4,2023-01-08,10:00:00,1012228,Yoga Tab 3,1,1,1,0,0,0,...,0,0,1,1,3,3,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,2023-01-14,21:00:00,9040283,NaN,1,1,1,32,0,32.400064,...,1,1,1,1,4,4,4,1,1,1
529,2023-01-14,22:00:00,1012228,NaN,1,1,1,1,0,1.473718,...,0,0,1,1,4,4,4,1,1,1
530,2023-01-14,22:00:00,1012479,NaN,1,1,1,2,0,2.482812,...,0,0,1,1,4,4,4,1,1,1
531,2023-01-14,22:00:00,9067777,NaN,1,0,1,0,0,0,...,0,0,1,1,2,2,2,1,1,1


In [1165]:
df = dfs_merged_fillNA[5]

print("Original_dtypes:")
print(df.dtypes)
 
df[df.columns[4:]] = df.iloc[:,4:].apply(pd.to_numeric)
 
print("New_dtypes:")
print(df.dtypes)

Original_dtypes:
dateFull                      object
time                          object
cityId                        object
mobileDeviceMarketingName     object
totalUsers                     int64
newUsers                       int64
activeUsers                    int64
userEngagementDuration         int64
scrolledUsers                  int64
averageSessionDuration       float64
bounceRate                   float64
engagedSessions                int64
engagementRate               float64
sessions                       int64
sessionsPerUser              float64
eventCount                     int64
eventCountPerUser            float64
eventsPerSession             float64
screenPageViews                int64
screenPageViewsPerSession    float64
screenPageViewsPerUser       float64
dtype: object
New_dtypes:
dateFull                      object
time                          object
cityId                        object
mobileDeviceMarketingName     object
totalUsers                     i

In [1168]:
df.to_excel("output.xlsx",index=False, date_format="yyyy-mm-dddd")

TypeError: NDFrame.to_excel() got an unexpected keyword argument 'date_format'

In [1167]:


# read the demo2.xlsx file
#df=pd.read_excel(fullFilenameWrite, engine='openpyxl')

# appending the data of df after the data of demo1.xlsx
with pd.ExcelWriter("output.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    df.to_excel(writer, sheet_name="Sheet1",header=None, startrow=writer.sheets["Sheet1"].max_row,index=False)